In [21]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import numpy as np
import os 
import requests
import math

### 1. Прочитайте цей набір даних пауерліфтингу в pandas DataFrame.

In [6]:
def download_document(file_name, document_url):
    if os.path.exists(file_name):
        pass
    else:
        response = requests.get(document_url)
        if response.status_code == 200:
            with open(file_name, 'wb') as f:
                f.write(response.content)
            return f    

        else:
            print(f'Failed to download the document. Status code: {response.status_code}')



file_name = 'ipf_lifts.csv'
document_url = 'https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2019/2019-10-08/'
download_document(file_name, document_url)

In [7]:
df = pd.read_csv(file_name)
df.head()

,name,sex,event,equipment,age,age_class,division,bodyweight_kg,weight_class_kg,best3squat_kg,best3bench_kg,best3deadlift_kg,place,date,federation,meet_name
0,Hiroyuki Isagawa,M,SBD,Single-ply,NaN,NaN,NaN,67.5,67.5,205.0,140.0,225.0,1,1985-08-03,IPF,World Games
1,David Mannering,M,SBD,Single-ply,24.0,24-34,NaN,67.5,67.5,225.0,132.5,235.0,2,1985-08-03,IPF,World Games
2,Eddy Pengelly,M,SBD,Single-ply,35.5,35-39,NaN,67.5,67.5,245.0,157.5,270.0,3,1985-08-03,IPF,World Games
3,Nanda Talambanua,M,SBD,Single-ply,19.5,20-23,NaN,67.5,67.5,195.0,110.0,240.0,4,1985-08-03,IPF,World Games
4,Göran Henrysson,M,SBD,Single-ply,NaN,NaN,NaN,67.5,67.5,240.0,140.0,215.0,5,1985-08-03,IPF,World Games
5,PJ Joseph,M,SBD,Single-ply,NaN,NaN,NaN,67.5,67.5,200.0,100.0,230.0,6,1985-08-03,IPF,World Games
6,Wilfried Dörner,M,SBD,Single-ply,32.5,24-34,NaN,67.5,67.5,220.0,140.0,235.0,7,1985-08-03,IPF,World Games


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41152 entries, 0 to 41151
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   name              41152 non-null  object 
 1   sex               41152 non-null  object 
 2   event             41152 non-null  object 
 3   equipment         41152 non-null  object 
 4   age               38246 non-null  float64
 5   age_class         38268 non-null  object 
 6   division          40525 non-null  object 
 7   bodyweight_kg     40965 non-null  float64
 8   weight_class_kg   41151 non-null  object 
 9   best3squat_kg     27454 non-null  float64
 10  best3bench_kg     38690 non-null  float64
 11  best3deadlift_kg  27124 non-null  float64
 12  place             41152 non-null  object 
 13  date              41152 non-null  object 
 14  federation        41152 non-null  object 
 15  meet_name         41152 non-null  object 
dtypes: float64(5), object(11)
memory usage: 

In [9]:
df.isnull().sum()

name                    0
sex                     0
event                   0
equipment               0
age                  2906
age_class            2884
division              627
bodyweight_kg         187
weight_class_kg         1
best3squat_kg       13698
best3bench_kg        2462
best3deadlift_kg    14028
place                   0
date                    0
federation              0
meet_name               0
dtype: int64

Можна було б заповнити незавопнені значння віку і вікової групи, але нажаль, в сеті є некорректні данні (для тої самої людини різниця у даті змагань і різниця у зазначеному віці на той мометн не співпадають).\
Також було б логічно видалити группу age_class 5-12 - там 1 учасник - більше сходе на помилку в 1(конкретно цьому) записі.\
weight_class_kg також можна було б доповнити на основі екстраполяції. 

### 2. Знайдіть рекорди кожної статі та кожного підрозділу в кожній із 3 вправ (жим лежачи, присідання та станова тяга).

In [12]:
df_2 = df[['sex', 'division', 'best3squat_kg', 'best3bench_kg', 'best3deadlift_kg']].copy()
df_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41151 entries, 0 to 41151
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   sex               41151 non-null  object 
 1   division          40524 non-null  object 
 2   best3squat_kg     27453 non-null  float64
 3   best3bench_kg     38689 non-null  float64
 4   best3deadlift_kg  27123 non-null  float64
dtypes: float64(3), object(2)
memory usage: 1.9+ MB


In [13]:
df_2 = df_2.groupby(['sex','division']).agg('max')
df_2

best3squat_kg  best3bench_kg  best3deadlift_kg
sex division                                                   
F   Heavy                305.0          192.5             248.5
    Juniors              282.5          190.5             252.5
    Light                220.0          150.0             202.5
    Masters 1            245.0          197.5             248.0
    Masters 2            255.0          172.5             227.5
    Masters 3            210.0          140.0             190.0
    Masters 4            135.0          115.0             142.5
    Middle               247.5          170.0             235.0
    Open                 322.5          235.0             270.5
    Sub-Juniors          265.0          170.0             238.0
    Super                267.5          190.0             245.0
    SuperHeavy           310.5          205.0             247.5
M   Heavy                432.0          330.0             382.5
    Juniors              450.0          375.0             377.5
    Light                325.5          217.5             315.0
    Masters 1            405.0          345.0             395.0
    Masters 2            365.0          310.5             335.0
    Masters 3            300.0          260.0             305.0
    Masters 4            235.0          220.0             260.5
    Middle               370.0          268.5             345.0
    Open                 490.0          415.0             407.5
    Sub-Juniors          370.0          310.0             335.0
    Super                465.0          360.0             387.5
    SuperHeavy           475.0          405.0             420.0

### 3. Порахуйте кількість перемог кожного учасника, беручи за перемогу 1-е місце в конкретному дивізіоні

In [14]:
df_winners = df[['name', 'division', 'place']][df['place']=='1'].copy()
df_winners['place'].value_counts()

1    6480
Name: place, dtype: int64

In [15]:
df_winners = df_winners.groupby(['name','division']).agg('count')

In [16]:
df_winners

place
name                      division          
A Ernandos-Ortega         Sub-Juniors      1
A. Raface                 Masters 1        1
Aarre Käpylä              Juniors          1
Ab Wolders                Masters 3        2
                          Open             1
...                                      ...
Øyvind Bjørnsen           Juniors          1
Þórunn Brynja Jónasdóttir Open             1
Štefan Koľšovský          Masters 1        5
                          Masters 2        2
Štěpánka Židová           Open             1

[3881 rows x 1 columns]

### 4. Використайте DataFrame з вправи 3, щоб показати для кожної комбінації підрозділу та статі, учасника з найбільшою кількістю перемог серед учасників, які будь-коли брали участь у цьому дивізіоні.

In [17]:
df_4 = pd.merge(df, df_winners, how = 'left', on=['name','division'])
df_4['max_winns']=df_4.groupby(['division'])['place_y'].transform(max)
df_4 = df_4[df_4['place_y'] == df_4['max_winns']]

In [18]:
df_4 = df_4[['division', 'name', 'sex', 'max_winns']].drop_duplicates(subset=['division', 'sex', 'name']).sort_values(by=['max_winns'], ascending = False)

In [19]:
df_4

,division,name,sex,max_winns
5731,Open,Sergey Fedosienko,M,20.0
2703,Open,Ielja Strik,F,20.0
5179,Masters 2,Hana Takáčová,F,10.0
6803,Masters 2,Hideaki Inaba,M,10.0
7967,Masters 1,Eric Kupperstein,M,9.0
6416,Masters 1,Priscilla Ribic,F,9.0
2388,Sub-Juniors,Daniela Kolesnik,F,8.0
2674,Masters 4,Bill Helmich,M,8.0
4148,Masters 3,Hiraku Fushimi,M,8.0
5202,Masters 4,Bebe Burns,F,8.0
